In [10]:
import os
import re
import json
import glob
import pandas as pd

pd.set_option('future.no_silent_downcasting', True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None) 

### Define function

In [11]:
def clean_data(filename):
    '''Load dataset from the specified filename, clean it by dropping duplicates, filtering for Singapore.'''
    df = pd.read_csv(filename, low_memory=False)

    # Replace unicode
    df = df.replace('\u202f', ' ', regex=True).replace('\u2013', '-', regex=True)

    # Drop duplicates
    df.drop_duplicates(subset=['address'], inplace=True)

    # Filter out rows where country is not in SG
    df = df[df['complete_address'].astype(str).str.contains('"country":"SG"', na=False)]

    # Drop unused columns
    df = drop_columns(df)

    return df

def drop_missing_open_hours(df):
    '''Filter out rows where open_hours is not available.'''
    return df[df['open_hours'].astype(str) != "{}"]

def drop_low_ratings(df, threshold=1):
    '''Filter out rows where ratings are below a certain threshold.'''
    return df[df['rating'].astype(float) >= threshold]

def drop_columns(df):
    '''Drop unused columns from the dataframe.'''
    df.drop(columns=[
        'input_id', 
        'popular_times', 
        'plus_code',
        # 'reviews_per_rating',
        'cid',
        'status',
        'reviews_link',
        'thumbnail',
        'timezone',
        'data_id',
        'reservations',
        'order_online',
        'menu',
        'owner',
        # 'address',
        'user_reviews',
        'user_reviews_extended',
        'emails',
        ], inplace=True, errors='ignore')
    return df

def combine_dataframes(dfs):
    '''Combine multiple dataframes into one, dropping duplicates based on the 'address' column.'''
    combined = pd.concat(dfs, ignore_index=True)
    num_duplicates = combined['address'].duplicated().sum()
    combined = combined.drop_duplicates(subset=['address'])
    print(f"duplicate rows: {num_duplicates}")
    return combined

def print_unique_categories(df, exclude_keyword=None):
    '''
    Print unique categories from the dataframe.
    Optionally exclude categories containing one or more keywords.
    '''
    unique_categories = df['category'].unique()

    if exclude_keyword:
        exclude_keyword = [kw.lower() for kw in exclude_keyword]
        unique_categories = [
            c for c in unique_categories
            if all(kw not in str(c).lower() for kw in exclude_keyword)
        ]

    print("Unique categories:")
    for category in unique_categories:
        print(f" - {category}")

def to_csv(df, filename):
    '''Save the dataframe to a CSV file.'''
    if not df.empty:
        df.to_csv(filename, index=False)
    else:
        print(f"No data to save to {os.path.basename(filename)}.")

def drop_low_ratings(df, threshold=1):
    '''Filter out rows where ratings are below a certain threshold.'''
    return df[df['review_rating'].astype(float) >= threshold]

def drop_low_reviews(df, threshold=100):
    '''Filter out rows where number of reviews are below a certain threshold.'''
    return df[df['review_count'].astype(float) >= threshold]

def remove_street_view(images):
    if not isinstance(images, (list, str)):
        return images

    try:
        image_json = json.loads(images)
        filtered_images = [img for img in image_json if not any(keyword in img.get('title', '').lower() for keyword in ['street view', '360'])]
        return filtered_images
    except (json.JSONDecodeError, TypeError):
        return images
    
def map_price(price):
    if pd.isna(price):
        return None

    price = str(price).strip()

    # Direct symbolic mapping
    symbol_map = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}
    if price in symbol_map:
        return symbol_map[price]

    # Extract numbers
    nums = re.findall(r'\d+', price)
    if not nums:
        return None

    nums = [int(n) for n in nums]
    mid = sum(nums) / len(nums)

    # Determine price level
    if mid < 20:
        return 1
    elif 20 <= mid <= 50:
        return 2
    elif 50 < mid <= 100:
        return 3
    else:
        return 4
    
def clean_images_field(images_raw):
    """Extract all image URLs from the images field."""
    if isinstance(images_raw, list):
        # When it's already a list of dicts
        return [
            item["image"]
            for item in images_raw
            if isinstance(item, dict) and "image" in item and isinstance(item["image"], str)
        ]
    return []
    
def clean_videos_field(videos_raw):
    """Clean and normalize video URLs."""
    if not isinstance(videos_raw, str) or not videos_raw.strip():
        return []
    urls = re.findall(r'https://[^,\s]+', videos_raw)
    cleaned = []
    for url in urls:
        url = url.split("|")[0]           # remove resolution part
        if "=mm" in url:
            url = url.split("=mm")[0]     # remove anything after =mm
        cleaned.append(url)
    return cleaned

def create_flags(row, keywords):
    """Return True if any of the given keywords appear in option names with enabled=True."""
    if pd.isna(row):
        return False

    data = json.loads(row) if isinstance(row, str) else row

    for cat in data:
        for opt in cat.get('options', []):
            opt_name = opt.get('name', '').lower()
            enabled = opt.get('enabled', False)
            if enabled and any(kw.lower() in opt_name for kw in keywords):
                return True
    return False

def remove_about(row, category_name):
    '''Remove the category name from the about field if it appears at the start.'''
    if pd.isna(row) or pd.isna(category_name):
        return row

    data = json.loads(row) if isinstance(row, str) else row
    filtered = [cat for cat in data if cat.get('name') != category_name]
    return json.dumps(filtered) if isinstance(row, str) else filtered

### Concatenate all data

In [12]:
INPUT_DIR = '../sg/'
OUTPUT_DIR = '../output/'

# Read all CSV files in the folder
csv_files = glob.glob(os.path.join(INPUT_DIR, "*.csv"))

# Clean and combine data
dataframes = [clean_data(file) for file in csv_files]
all_data = combine_dataframes(dataframes)

# print(all_data.dtypes)


duplicate rows: 1935


In [13]:
# Clean data
all_data = all_data.rename(columns={"title": "name", "raw_images": "videos"})
all_data['images'] = all_data['images'].apply(remove_street_view)
all_data['price_range'] = all_data['price_range'].apply(map_price)
all_data = all_data.rename(columns={"price_range": "price_level"})
all_data['complete_address'] = all_data['complete_address'].str.replace('\\u0026', "&", regex=False)

# Handle images & videos
all_data['images'] = all_data['images'].apply(clean_images_field)
all_data['videos'] = all_data['videos'].apply(clean_videos_field)

# Drop low rating rows
all_data = drop_low_ratings(all_data, threshold=1)

# Create flags for kids & pets friendly
all_data['kids_friendly'] = all_data['about'].apply(create_flags, keywords=['Good for kids'])
all_data['dogs_friendly'] = all_data['about'].apply(create_flags, keywords=['Dogs allowed', 'Dogs allowed inside', 'Dogs allowed outside'])
all_data['wheelchair_rental'] = all_data['about'].apply(create_flags, keywords=['Wheelchair rental'])
all_data['wheelchair_accessible_car_park'] = all_data['about'].apply(create_flags, keywords=['Wheelchair accessible car park'])
all_data['wheelchair_accessible_entrance'] = all_data['about'].apply(create_flags, keywords=['Wheelchair accessible entrance'])
all_data['wheelchair_accessible_seating'] = all_data['about'].apply(create_flags, keywords=['Wheelchair accessible seating'])
all_data['wheelchair_accessible_toilet'] = all_data['about'].apply(create_flags, keywords=['Wheelchair accessible toilet'])
all_data['halal_food'] = all_data['about'].apply(create_flags, keywords=['Halal food'])
all_data['vegan_options'] = all_data['about'].apply(create_flags, keywords=['Vegan options'])
all_data['vegetarian_options'] = all_data['about'].apply(create_flags, keywords=['Vegetarian options'])
all_data['reservations_required'] = all_data['about'].apply(create_flags, keywords=['Reservations required'])
all_data['hiking'] = all_data['about'].apply(create_flags, keywords=['Hiking', 'Point-to-point trail', 'Trail difficulty'])
all_data['cycling'] = all_data['about'].apply(create_flags, keywords=['Cycling'])

# Remove certain "about" field
all_data['about'] = all_data['about'].apply(remove_about, category_name="Atmosphere")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Amenities")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Dining options")
all_data['about'] = all_data['about'].apply(remove_about, category_name="From the business")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Getting here")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Offerings")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Parking")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Payments")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Pets")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Popular for")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Recycling")
all_data['about'] = all_data['about'].apply(remove_about, category_name="Service options")

to_csv(all_data, f"{OUTPUT_DIR}poi.csv")

In [14]:
categories = {}

for row in all_data['about'].dropna():
    data = json.loads(row) if isinstance(row, str) else row

    for category in data:
        cat_name = category.get('name')
        opts = [opt.get('name') for opt in category.get('options', [])]

        # keep the one with more options if duplicate category appears
        if cat_name not in categories or len(opts) > len(categories[cat_name]):
            categories[cat_name] = opts
    
# print results
for cat_name, opts in sorted(categories.items()):
    print(f"{cat_name}:")
    for opt in sorted(opts):
        print(f"  - {opt}")
    print()


Accessibility:
  - Assistive hearing loop
  - Wheelchair rental
  - Wheelchair-accessible car park
  - Wheelchair-accessible entrance
  - Wheelchair-accessible seating
  - Wheelchair-accessible toilet

Activities:
  - Birdwatching
  - Cycling
  - Hiking
  - Jogging
  - Point-to-point trail
  - Trail difficulty
  - Walking

Children:
  - Discounts for kids
  - Family discount
  - Good for kids
  - Has changing table(s)
  - Kid-friendly activities
  - Nursing room

Crowd:
  - Family friendly
  - Groups
  - LGBTQ+ friendly
  - Tourists
  - Transgender safe space
  - University students

Highlights:
  - Fireplace
  - Great beer selection
  - Great cocktails
  - Great coffee
  - Great dessert
  - Great tea selection
  - Great wine list
  - Rooftop seating
  - Sport

Planning:
  - Accepts reservations
  - Brunch reservations recommended
  - Dinner reservations recommended
  - Lunch reservations recommended
  - Reservations required
  - Usually a wait



### Michelin

In [15]:
import ast

# Fill empty price_level using Michelin data
michelin = pd.read_csv(f"{OUTPUT_DIR}michelin.csv")
michelin['price'] = michelin['price'].apply(map_price)
price_lookup = michelin.set_index("name")["price"]
all_data["price_level"] = all_data["price_level"].fillna(all_data["name"].map(price_lookup))

# Fill empty description with Michelin data
description_lookup = michelin.set_index("name")["description"]
all_data["descriptions"] = all_data["descriptions"].fillna(all_data["name"].map(description_lookup))

# Insert michelin images into images field by adding into the list

def to_list(s):
    if isinstance(s, list):
        return s
    if not s or not isinstance(s, str):
        return []
    try:
        return ast.literal_eval(s)
    except Exception:
        return []

# Combine image lists and remove duplicates
def merge_images(row):
    imgs_all = to_list(row['images'])
    imgs_michelin = to_list(row['michelin_images'])
    combined = list(dict.fromkeys(imgs_all + imgs_michelin))  # preserves order, removes duplicates
    return str(combined)

# Assuming `michelin['images']` aligns by index
all_data['michelin_images'] = michelin['images']
all_data['images'] = all_data.apply(merge_images, axis=1)
all_data.drop(columns=['michelin_images'], inplace=True)

to_csv(all_data, f"{OUTPUT_DIR}poi.csv")